# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [110]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [111]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In [112]:
# temp = np.arange(0,30,2)
# print(len(temp), temp)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

### <font color='green'>TODO - batch_size </font>
- 64

##### <font color='red'> Path to train and test csv's => how to submit?</font>

In [113]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 16

In [114]:
print(len(train_doc), len(val_doc))

print(train_doc[0])
import scipy
print(tf.__version__)
print(scipy.__version__)

663 100
WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1

1.10.0
1.1.0


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

### <font color='green'>TODO - select the list of image indexes from the video
- take +/-7 images from the mid position </font>

### <font color='green'>TODO - num_batches formula 
    
- x = len(img_idx)
- y = height of image
- z = width of image

In [115]:
image_width = 100
image_height = 100
# image_idx = list(np.arange(0,30,2))
image_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
print(len(image_idx))

18


In [116]:
num_batches = len(train_doc) // batch_size
        
# for remaining videos
remaining_sequences = len(train_doc) % batch_size

print("batch_size = " , batch_size , " and sequences = " , remaining_sequences, num_batches)

print(list(range(num_batches)))

batch_size =  16  and sequences =  7 41
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]


In [117]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # img_idx = #create a list of image numbers you want to use for a particular video
    img_idx = image_idx
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size
        
        # for remaining videos
        remaining_sequences = len(folder_list) % batch_size
#         print("batch_size = " , batch_size, " and sequences = " , remaining_sequences)
        x = len(img_idx)
        y = image_height
        z = image_width        
        
        for batch in range(num_batches): # we iterate over the number of batches   
#             print("current batch = ", batch)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if(image.shape[0]!=image.shape[1]):
                        image=image[0:120,20:140]
                    
                    # `imresize` is deprecated!
                    # `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
                    # Use ``skimage.transform.resize`` instead.
                    image = imresize(image, (y,z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    # after normalization, the values will be converted to float, is that fine?
                    epsilon = 1e-7
                    norm_image = (image - image.mean(axis=(0,1,2), keepdims=True)) / image.std(axis=(0,1,2), keepdims=True)
                    
                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0]
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
#         print("processing remaining sequences = " ,remaining_sequences, "::num_batches=", num_batches)
        if remaining_sequences != 0:
            batch_size = remaining_sequences
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if(image.shape[0]!=image.shape[1]):
                        image=image[0:120,20:140]
                    # `imresize` is deprecated!
                    # `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
                    # Use ``skimage.transform.resize`` instead.
                    image = imresize(image, (y,z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    # after normalization, the values will be converted to float, is that fine?
                    epsilon = 1e-5
                    # norm_image = (image - image.mean(axis=(0,1,2), keepdims=True)) / image.std(axis=(0,1,2), keepdims=True)+epsilon
                    norm_image = (image - image.mean(axis=(0,1,2), keepdims=True)) / image.std(axis=(0,1,2), keepdims=True)

                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0]
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1]
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2]

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
                yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [118]:
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'

train_path = './Project_data/train'
val_path = './Project_data/val'

### <font color='green'> TODO - choose Epochs

In [119]:
curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 #50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [120]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers


#write your model here

## 3d CNN models

- https://ieeexplore.ieee.org/document/8545718 (later experiment with more conv)

In [121]:
# filter_size = [8,16,32,64,128,256]
filter_size = [8,16,32,64,128,256]
dense_layer_size = [256,128] #256,256

input_shape=(len(image_idx),image_width,image_height,3)

model = Sequential()


#Layer 1
model.add(Conv3D(filters=filter_size[0],kernel_size=(3,3,3),input_shape=input_shape, padding='same')) # 'valid'
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))


#Layer 2
model.add(Conv3D(filters=filter_size[1],kernel_size=(3,3,3), padding='same')) # 'valid'
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))


#Layer 3
model.add(Conv3D(filters=filter_size[2],kernel_size=(1,3,3), padding='same')) # 'valid'
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Dropout(0.25))

# #Layer 4
model.add(Conv3D(filters=filter_size[3],kernel_size=(1,3,3), padding='same')) # 'valid'
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Dropout(0.25))


#Flatten Layers
model.add(Flatten())

# Dense 1
model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.5))

# Dense 2
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.5))

# final softmax
#softmax layer
model.add(Dense(5, activation='softmax'))

### Convolutions + RNN
- The conv2D network will extract a feature vector for each image, and a sequence of these feature vectors is then fed to an RNN-based network. The output of the RNN is a regular softmax (for a classification problem such as this one).

#### There are a few key things to note about the conv-RNN architecture:

- You can use transfer learning in the 2D CNN layer rather than training your own CNN 
- GRU can be a better choice than an LSTM since it has lesser number of gates (and thus parameters)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [122]:
from keras.optimizers import Adam

In [123]:
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 18, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_25 (Batc (None, 18, 100, 100, 8)   32        
_________________________________________________________________
activation_25 (Activation)   (None, 18, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 9, 50, 50, 8)      0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 9, 50, 50, 16)     3472      
_________________________________________________________________
batch_normalization_26 (Batc (None, 9, 50, 50, 16)     64        
_________________________________________________________________
activation_26 (Activation)   (None, 9, 50, 50, 16)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [124]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [128]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
#early stopping required
ES = EarlyStopping(monitor='val_loss', verbose=1, patience=3)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)
# patience can be increased with early stopping (5)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
# callbacks_list = [checkpoint, LR, ES]
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [129]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [130]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


42/42 [==============================] - 46s 1s/step - loss: 12.4876 - categorical_accuracy: 0.2048 - val_loss: 12.6642 - val_categorical_accuracy: 0.2143

Epoch 00001: val_loss improved from inf to 12.66422, saving model to model_init_2019-09-0721_47_26.764867/model-00001-12.48757-0.20476-12.66422-0.21429.h5
Epoch 2/20
42/42 [==============================] - 43s 1s/step - loss: 11.9186 - categorical_accuracy: 0.2476 - val_loss: 13.8155 - val_categorical_accuracy: 0.1429

Epoch 00002: val_loss did not improve from 12.66422
Epoch 3/20
26/42 [=================>............] - ETA: 18s - loss: 13.0923 - categorical_accuracy: 0.1692

c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


42/42 [==============================] - 38s 893ms/step - loss: 12.7287 - categorical_accuracy: 0.1761 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00003: val_loss improved from 12.66422 to 12.08857, saving model to model_init_2019-09-0721_47_26.764867/model-00003-12.83315-0.17416-12.08857-0.25000.h5
Epoch 4/20
42/42 [==============================] - 28s 666ms/step - loss: 13.3002 - categorical_accuracy: 0.1667 - val_loss: 11.5129 - val_categorical_accuracy: 0.2857

Epoch 00004: val_loss improved from 12.08857 to 11.51293, saving model to model_init_2019-09-0721_47_26.764867/model-00004-13.30021-0.16667-11.51293-0.28571.h5
Epoch 5/20
42/42 [==============================] - 27s 647ms/step - loss: 11.8905 - categorical_accuracy: 0.2460 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00005: val_loss did not improve from 11.51293
Epoch 6/20
42/42 [==============================] - 27s 644ms/step - loss: 11.9584 - categorical_accuracy: 0.2302 - val_loss: 11.